In [89]:
import tensorflow as tf
from tensorflow.keras.layers import Dense,Flatten,Conv2D,MaxPooling2D

In [90]:
Cifar10 = tf.keras.datasets.cifar10

(x_train, y_train), (x_test, y_test) = Cifar10.load_data()
x_train = x_train / 255.0
x_test = x_test / 255.0

#x_train =  x_train [...,tf.newaxis]
#x_test =  x_test [...,tf.newaxis]

In [91]:
x_train[0].shape

(32, 32, 3)

In [92]:
type(x_train[0][0,0,0])

numpy.float64

In [93]:
train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(1000).batch(32)
test_data = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

In [94]:
class Model(tf.keras.Model):
  def __init__(self):
    super().__init__()
    self.conv1 = Conv2D(32,3,activation="relu")
    self.max1 = MaxPooling2D(2,2)
    self.conv2 = Conv2D(64,3,activation="relu")
    self.max2 = MaxPooling2D(2,2)
    self.conv3 = Conv2D(64,3,activation="relu")
    self.flatten = Flatten()
    self.dense1 = Dense(128,activation="relu")
    self.dense2 = Dense(10)

  def call(self,x):
     x = self.conv1(x)
     x = self.max1(x)
     x = self.conv2(x)
     x = self.max2(x)
     x = self.conv3(x)
     x = self.flatten(x)
     x = self.dense1(x)
     x = self.dense2(x)
     return x

model = Model()

In [95]:
loss_function = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam()

In [96]:
train_loss = tf.keras.metrics.Mean()
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()

test_loss = tf.keras.metrics.Mean()
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()

In [97]:
@tf.function
def train(imagse,labels):
    with tf.GradientTape() as tape:
        pred = model(imagse)
        loss = loss_function(labels,pred)
    gradients = tape.gradient(loss,model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels,pred)

In [98]:
@tf.function
def test(imagse,labels):
        pred = model(imagse)
        loss = loss_function(labels,pred)
        test_loss(loss)
        test_accuracy(labels,pred)

In [99]:
epochs = 10
for epoch in range(epochs):
  train_loss.reset_state()
  train_accuracy.reset_state()
  test_loss.reset_state()
  test_accuracy.reset_state()

  for imagse , labels in train_data:
    train(imagse,labels)

  for imagse , labels in test_data:
    test(imagse,labels)

  print("epoch :" ,epoch + 1,
        f"Train Loss :{train_loss.result()}",
        f"Train accuracy:{train_accuracy.result()}",
        f"Test Loss :   {test_loss.result()}",
        f"Test accuracy:{test_accuracy.result()}")


epoch : 1 Train Loss :1.5132251977920532 Train accuracy:0.44811999797821045 Test Loss :   1.2091113328933716 Test accuracy:0.565500020980835
epoch : 2 Train Loss :1.1456936597824097 Train accuracy:0.5945000052452087 Test Loss :   1.0826209783554077 Test accuracy:0.6140999794006348
epoch : 3 Train Loss :0.9934507012367249 Train accuracy:0.6492999792098999 Test Loss :   0.9729187488555908 Test accuracy:0.6621000170707703
epoch : 4 Train Loss :0.8972755074501038 Train accuracy:0.6852399706840515 Test Loss :   0.9243831038475037 Test accuracy:0.6850000023841858
epoch : 5 Train Loss :0.8235049247741699 Train accuracy:0.7100399732589722 Test Loss :   0.9506688714027405 Test accuracy:0.6710000038146973
epoch : 6 Train Loss :0.7613471746444702 Train accuracy:0.7299799919128418 Test Loss :   0.8755048513412476 Test accuracy:0.697700023651123
epoch : 7 Train Loss :0.7103512287139893 Train accuracy:0.7498599886894226 Test Loss :   0.8975512385368347 Test accuracy:0.6955000162124634
epoch : 8 Trai

In [104]:
import cv2
import numpy as np

classes_name = ["airplane", "cars", "birds", "cats", "deer", "dogs", "frogs", "horses", "ships","trucks"]
image = cv2.imread("image2.jpeg")
image = cv2.resize(image,(32,32))
image = image [tf.newaxis,...]
image = image.astype("float32")

prad = model(image)
result = np.argmax(prad)
result = classes_name[result]
print(result)

trucks


In [ ]:
model.export("my_model2")
